In [1]:
import gymnasium as gym
import ray
from ray import tune

from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.connectors.env_to_module import FlattenObservations


# 1. Ray initialisieren
ray.init(ignore_reinit_error=True)

2025-07-29 16:07:17,960	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-07-29 16:07:18,317	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-07-29 16:07:20,969	INFO worker.py:1927 -- Started a local Ray instance.


Python version:,3.10.18
Ray version:,2.48.0


In [3]:
env_name = "Taxi-v3"

In [28]:
# Configure the algorithm.
config = (
    PPOConfig()
    .environment(env=env_name)
    .framework("torch")
    .env_runners(
        num_env_runners=2,
        # Observations are discrete (ints) -> We need to flatten (one-hot) them.
        env_to_module_connector=lambda env: FlattenObservations(),
    )
    .evaluation(evaluation_num_env_runners=1)
    .resources(num_gpus=1)
)

# Build the algorithm.
algo = config.build_algo()

2025-07-29 16:41:12,255	WARNING algorithm_config.py:4921 -- You have specified 1 evaluation workers, but your `evaluation_interval` is 0 or None! Therefore, evaluation doesn't occur automatically with each call to `Algorithm.train()`. Instead, you have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
[2025-07-29 16:41:12,286 E 125052 125052] core_worker.cc:2740: Actor with class name: 'SingleAgentEnvRunner' and ID: 'ef9cccc00f18fdbbf162b8d001000000' has constructor arguments in the object store and max_restarts > 0. If the arguments in the object store go out of scope or are lost, the actor restart will fail. See https://github.com/ray-project/ray/issues/53727 for more details.
[2025-07-29 16:41:12,314 E 125052 125052] core_worker.cc:2740: Actor with class name: 'SingleAgentEnvRunner' and ID: '6c43182a9bf55a48e2620a6c01000000' has constructor arguments in the object store and max_restarts > 0. If the arguments in the object store go out of scope or are los

In [31]:
# Training
for i in range(3):
    result = algo.train()
    print(f"Iteration {i} ")
    pprint(result)

Iteration 0 
{'config': {'_disable_action_flattening': False,
            '_disable_execution_plan_api': -1,
            '_disable_initialize_loss_from_dummy_batch': False,
            '_disable_preprocessor_api': False,
            '_dont_auto_sync_env_runner_states': False,
            '_enable_rl_module_api': -1,
            '_env_to_module_connector': <function <lambda> at 0x723958b9c3a0>,
            '_fake_gpus': False,
            '_is_atari': None,
            '_is_online': True,
            '_learner_class': None,
            '_learner_connector': None,
            '_model_config': {},
            '_module_to_env_connector': None,
            '_per_module_overrides': {},
            '_prior_exploration_config': {'type': 'StochasticSampling'},
            '_rl_module_spec': None,
            '_tf_policy_handles_more_than_one_loss': False,
            '_torch_grad_scaler_class': None,
            '_torch_lr_scheduler_classes': None,
            '_train_batch_size_per_learner': N

Iteration 1 
{'config': {'_disable_action_flattening': False,
            '_disable_execution_plan_api': -1,
            '_disable_initialize_loss_from_dummy_batch': False,
            '_disable_preprocessor_api': False,
            '_dont_auto_sync_env_runner_states': False,
            '_enable_rl_module_api': -1,
            '_env_to_module_connector': <function <lambda> at 0x723958b9c3a0>,
            '_fake_gpus': False,
            '_is_atari': None,
            '_is_online': True,
            '_learner_class': None,
            '_learner_connector': None,
            '_model_config': {},
            '_module_to_env_connector': None,
            '_per_module_overrides': {},
            '_prior_exploration_config': {'type': 'StochasticSampling'},
            '_rl_module_spec': None,
            '_tf_policy_handles_more_than_one_loss': False,
            '_torch_grad_scaler_class': None,
            '_torch_lr_scheduler_classes': None,
            '_train_batch_size_per_learner': N

Iteration 2 
{'config': {'_disable_action_flattening': False,
            '_disable_execution_plan_api': -1,
            '_disable_initialize_loss_from_dummy_batch': False,
            '_disable_preprocessor_api': False,
            '_dont_auto_sync_env_runner_states': False,
            '_enable_rl_module_api': -1,
            '_env_to_module_connector': <function <lambda> at 0x723958b9c3a0>,
            '_fake_gpus': False,
            '_is_atari': None,
            '_is_online': True,
            '_learner_class': None,
            '_learner_connector': None,
            '_model_config': {},
            '_module_to_env_connector': None,
            '_per_module_overrides': {},
            '_prior_exploration_config': {'type': 'StochasticSampling'},
            '_rl_module_spec': None,
            '_tf_policy_handles_more_than_one_loss': False,
            '_torch_grad_scaler_class': None,
            '_torch_lr_scheduler_classes': None,
            '_train_batch_size_per_learner': N

In [32]:
# ... and evaluate it.
pprint(algo.evaluate())

# Release the algo's resources (remote actors, like EnvRunners and Learners).
algo.stop()

{'env_runners': {'agent_episode_return_mean': {'default_agent': -515.0},
                 'env_reset_timer': 0.000413251000281889,
                 'env_step_timer': 6.971735531206349e-05,
                 'env_to_module_connector': {'connector_pipeline_timer': 0.0001866496132408603,
                                             'timers': {'connectors': {'add_observations_from_episodes_to_batch': 7.363182864581167e-06,
                                                                       'add_states_from_episodes_to_batch': 4.529063367132337e-06,
                                                                       'add_time_dim_to_batch_and_zero_pad': 6.775441536417815e-06,
                                                                       'batch_individual_items': 1.920679267953981e-05,
                                                                       'flatten_observations': 4.268484275914268e-05,
                                                                       'numpy

In [13]:
algo??

In [14]:
import gymnasium as gym

# Test-Environment vorbereiten
env = gym.make("Taxi-v3")
obs, _ = env.reset()
done = False
total_reward = 0

# Zugriff auf Policy über den Worker
policy = algo.workers.local_worker().policy_map["default_policy"]

while not done:
    action = policy.compute_single_action(obs)
    obs, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated
    total_reward += reward
    env.render()

print(f"Total reward in test episode: {total_reward}")


AttributeError: 'function' object has no attribute 'local_worker'

2025-07-29 16:38:43,568	INFO worker.py:1765 -- Calling ray.init() again after it has already been called.
[2025-07-29 16:38:43,615 E 125052 125052] core_worker.cc:2740: Actor with class name: 'RolloutWorker' and ID: '32066a3ea73cbb53063d25f201000000' has constructor arguments in the object store and max_restarts > 0. If the arguments in the object store go out of scope or are lost, the actor restart will fail. See https://github.com/ray-project/ray/issues/53727 for more details.
[2025-07-29 16:38:43,664 E 125052 125052] core_worker.cc:2740: Actor with class name: 'RolloutWorker' and ID: 'd21aa701d5948e7b34406da701000000' has constructor arguments in the object store and max_restarts > 0. If the arguments in the object store go out of scope or are lost, the actor restart will fail. See https://github.com/ray-project/ray/issues/53727 for more details.
2025-07-29 16:38:46,771	WARNING util.py:61 -- Install gputil for GPU system monitoring.


Iteration 0
Iteration 1
Iteration 2


TypeError: unhashable type: 'list'

In [ ]:
git config --global user.email "nerdvsn@gmail.com"
ssh-keygen -t ed25519 -C "nerdvsn@gmail.com"
